In [1]:
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using QPControl
using LCPSim
using MeshCat
using MeshCatMechanisms
using LearningMPC
using LearningMPC.Models
using RigidBodySim
using Gurobi
using SimpleQP

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/QPControl.ji for module QPControl.
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LCPSim.ji for module LCPSim.


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


In [2]:
vis = Visualizer()
open(vis)

Listening on 0.0.0.0:8700...


INFO: Serving MeshCat visualizer at http://127.0.0.1:8700


In [3]:
Δt = 0.003

0.003

In [24]:
urdf = Pkg.dir("RigidBodyDynamics", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(Float64, urdf)
effort_bounds(findjoint(mechanism, "shoulder")) .= RigidBodyDynamics.Bounds(0, 0)
delete!(vis)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf), vis)

MeshCatMechanisms.MechanismVisualizer{RigidBodyDynamics.MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Fixed{Float64}},1},Array{RigidBodyDynamics.Joint{Float64,RigidBodyDynamics.Revolute{Float64}},1}},2}},MeshCat.Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat, 3)

In [25]:
function init!(x::MechanismState)
    zero!(x)
    set_configuration!(x, [π, 0])
    x
end

init! (generic function with 1 method)

In [62]:
Q = diagm([10, 0.1, 1, 1])
R = diagm([1e-4, 1e-4])
lqrsol = LearningMPC.LQRSolution(init!(MechanismState(mechanism)),
    Q, R, Δt)

(::LQRSolution) (generic function with 2 methods)

In [45]:
state = init!(MechanismState(mechanism))
set_velocity!(state, [0.1, 0])
problem = LearningMPC.simulation_problem(state, lqrsol, Δt, 5.0)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

In [123]:
mpc = QPControl.MPCController{QPControl.ContactPoint{4}}(
    mechanism,
    GurobiOptimizer(OutputFlag=0, TimeLimit=1)
)
stage = addstage!(mpc, Δt)

objective = 0
for stage in mpc.stages
    x̄ = @expression vcat(stage.q, stage.v) - lqrsol.x0
#     ū = @expression stage.u - lqrsol.u0
    ū = stage.v̇
    objective = @expression objective + x̄' * lqrsol.Q * x̄ + ū' * lqrsol.R * ū
end
x̄ = @expression vcat(last(mpc.stages).q, last(mpc.stages).v) - lqrsol.x0
objective = @expression objective + x̄' * lqrsol.S * x̄;
@objective mpc.qpmodel Minimize objective

Academic license - for non-commercial use only


In [127]:
state = init!(MechanismState(mechanism))
set_velocity!(state, [0.2, 0.1])
problem = LearningMPC.simulation_problem(state, mpc, Δt, 5.0; damping_kd=1e-2)

sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6)
setanimation!(mvis, sol)

true

INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
INFO: ws|server received OPCODE_PING
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")